# 라이브러리 설치 및 임포트

In [ ]:
# 1. Google 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 2. 필요한 라이브러리 설치
!pip install pydub

In [ ]:
# 3. 라이브러리 임포트
import os
import random
from pydub import AudioSegment
from openpyxl import Workbook

# 오디오 믹싱 함수

In [ ]:
def mix_audio_files(audio_files, output_path, silence_thresh=-50.0, target_length_ms=5000):
    """
    여러 오디오 파일을 믹싱하여 하나의 오디오 파일로 출력합니다.
    """
    first_audio = AudioSegment.from_file(audio_files[0])
    second_audio = AudioSegment.from_file(audio_files[1])
    mixed_audio = first_audio.overlay(second_audio)

    final_audio = mixed_audio[:target_length_ms]
    final_audio.export(output_path, format="wav")
    #print(f"{os.path.basename(output_path)} saved: mix {first_audio} and {second_audio}")

# 두 파일을 랜덤으로 선택하여 믹스
## 1. Animal+Animal

In [ ]:
import os
import random
from openpyxl import Workbook

def create_mixed_audio_files(base_path, exclude_folders, output_folder, num_files=8000, spreadsheet_path="file_paths.xlsx"):
    """
    랜덤으로 선택된 오디오 파일을 믹싱하여 지정된 수만큼 파일을 생성하고, 그 경로를 스프레드시트에 한 줄씩 기록합니다.
    """
    # 모든 폴더 필터링
    all_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    filtered_folders = [folder for folder in all_folders if folder not in exclude_folders]

    # 출력 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 스프레드시트 초기화
    wb = Workbook()
    ws = wb.active
    ws.append(["index", "first-file", "second-file"])  # 첫 번째 row 설정

    # 파일 생성
    for i in range(num_files):
        # 랜덤으로 폴더 두 개 선택
        random_folders = random.sample(filtered_folders, 2)

        # 각 폴더에서 파일 랜덤 선택
        selected_files = []
        selected_file_names = []  # 파일 이름 저장 리스트
        for folder in random_folders:
            folder_path = os.path.join(base_path, folder)
            all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
            if all_files:
                selected_file = random.choice(all_files)
                selected_files.append(os.path.join(folder_path, selected_file))
                selected_file_names.append(selected_file)  # 파일 이름 추가

        # 선택된 파일 이름 출력
        print(f"Animal_Animal_{i + 1}.wav saved: mix {selected_file_names}")

        # 파일 믹싱
        if len(selected_files) == 2:  # 두 개 파일 모두 유효할 때만 믹싱
            output_path = os.path.join(output_folder, f"Animal_Animal_{i + 1}.wav")
            mix_audio_files(selected_files, output_path=output_path)

            # 한 줄씩 스프레드시트에 기록 (인덱스와 파일 경로)
            ws.append([i + 1, f"{random_folders[0]}/{selected_file_names[0]}", f"{random_folders[1]}/{selected_file_names[1]}"])


    print(f"Spreadsheet saved at {'/content/drive/MyDrive/Dataset/MixedData/Animal_Animal_info.xlsx'}")
    return(wb)

In [ ]:
base_path = '/content/drive/MyDrive/Dataset/SingleData/도훈'  # 기본 경로
# exclude_folders = ['.ipynb_checkpoints', '도훈']  # 제외할 폴더
output_folder = '/content/drive/MyDrive/Dataset/MixedData/Human_Animal'  # 출력 폴더
num_files = 16000  # 생성할 믹싱 파일 수

wb = create_mixed_audio_files(base_path, exclude_folders, output_folder, num_files=num_files)

Streaming output truncated to the last 5000 lines.
Animal_Animal_11002.wav saved: mix ['7.산비둘기_71.wav', 'Dog_12.wav']
Animal_Animal_11003.wav saved: mix ['Sheep_6.wav', '5.멧돼지_53.wav']
Animal_Animal_11004.wav saved: mix ['Chicken_6.wav', 'Monkey_4.wav']
Animal_Animal_11005.wav saved: mix ['2.까마귀_95.wav', '1.고라니_129.wav']
Animal_Animal_11006.wav saved: mix ['7.산비둘기_39.wav', 'Sheep_1.wav']
Animal_Animal_11007.wav saved: mix ['8.소쩍새_58.wav', 'Cow_1.wav']
Animal_Animal_11008.wav saved: mix ['Cat_105.wav', '6.까치_53.wav']
Animal_Animal_11009.wav saved: mix ['4.곤충류_31.wav', '3.양서류_98.wav']
Animal_Animal_11010.wav saved: mix ['Frog_7.wav', 'Cow_4.wav']
Animal_Animal_11011.wav saved: mix ['Dog_40.wav', '2.까마귀_75.wav']
Animal_Animal_11012.wav saved: mix ['3.양서류_106.wav', 'Bird_14.wav']
Animal_Animal_11013.wav saved: mix ['Elephant_2.wav', '9.참새_208.wav']
Animal_Animal_11014.wav saved: mix ['2.까마귀_3.wav', '6.까치_89.wav']
Animal_Animal_11015.wav s

In [ ]:
file_path = "/content/drive/MyDrive/Dataset/MixedData/Animal_Animal_info.xlsx"  # 원하는 경로와 파일명으로 수정
wb.save(file_path)

## Animal+Human

In [ ]:
import os
import random
from openpyxl import Workbook

def create_mixed_audio_files(base_path, exclude_folder, output_folder, num_random_files=3, spreadsheet_path="file_paths.xlsx", start_index=6341):
    """
    특정 폴더의 각 음성 파일에 대해 나머지 폴더에서 중복되지 않는 랜덤 음성 파일을 선택해 믹싱하여 파일을 생성합니다.
    """
    # 특정 폴더 경로 설정
    specific_folder_path = os.path.join(base_path, exclude_folder)
    specific_files = [f for f in os.listdir(specific_folder_path) if os.path.isfile(os.path.join(specific_folder_path, f))]

    # 나머지 폴더 필터링
    all_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    other_folders = [folder for folder in all_folders if folder != exclude_folder]

    # 출력 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 스프레드시트 초기화
    wb = Workbook()
    ws = wb.active
    ws.append(["index", "specific-file", "random-file", "output-file"])  # 첫 번째 row 설정

    file_index = start_index

    # 특정 폴더의 파일 처리
    for specific_file in specific_files:
        specific_file_path = os.path.join(specific_folder_path, specific_file)

        # 이미 해당 이름으로 시작하는 파일이 있는지 확인
        existing_files = [f for f in os.listdir(output_folder) if f.startswith(os.path.splitext(specific_file)[0])]
        if existing_files:
            print(f"Skipping {specific_file}: existing files found in {output_folder}")
            continue

        # 랜덤 음성 파일 선택 및 믹싱
        used_files = set()  # 중복 방지를 위한 사용된 파일 추적
        for _ in range(num_random_files):
            # 랜덤 폴더에서 랜덤 파일 선택
            random_file_path, random_file = None, None
            attempts = 0
            while attempts < 10:  # 무한 루프 방지
                random_folder = random.choice(other_folders)
                random_folder_path = os.path.join(base_path, random_folder)
                random_files = [f for f in os.listdir(random_folder_path) if os.path.isfile(os.path.join(random_folder_path, f))]

                if random_files:
                    random_file = random.choice(random_files)
                    random_file_path = os.path.join(random_folder_path, random_file)

                    if random_file_path not in used_files:  # 중복 검사
                        used_files.add(random_file_path)
                        break
                attempts += 1

            if random_file_path is None:
                print(f"Warning: Could not find a unique random file for {specific_file}")
                continue

            # 파일 이름 출력
            output_file_name = f"{os.path.splitext(specific_file)[0]}_mixed_{file_index}.wav"
            output_path = os.path.join(output_folder, output_file_name)

            print(f"{output_file_name} saved: mix {specific_file} with {random_file}")

            # 파일 믹싱
            mix_audio_files([specific_file_path, random_file_path], output_path=output_path)

            # 한 줄씩 스프레드시트에 기록
            ws.append([file_index, specific_file, f"{random_folder}/{random_file}", output_file_name])
            file_index += 1

    # 스프레드시트 저장
    wb.save(spreadsheet_path)
    print(f"Spreadsheet saved at {spreadsheet_path}")

# mix_audio_files 함수는 직접 구현해야 합니다.
# 두 개의 오디오 파일을 믹싱하여 출력하는 로직을 작성하세요.

base_path = '/content/drive/MyDrive/Dataset/SingleData/'  # 기본 경로
exclude_folder = '도훈'  # 특정 폴더 이름
output_folder = '/content/drive/MyDrive/Dataset/MixedData/Human_Animal'  # 출력 폴더
spreadsheet_path = '/content/drive/MyDrive/Dataset/MixedData/Human_Animal_info.xlsx'

create_mixed_audio_files(
    base_path,
    exclude_folder,
    output_folder,
    num_random_files=4,
    spreadsheet_path=spreadsheet_path,
    start_index=6341
)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
02bo0305.wav_normalized_mixed_11120.wav saved: mix 02bo0305.wav_normalized.wav with 5.멧돼지_391.wav
430o030t.wav_normalized_mixed_11121.wav saved: mix 430o030t.wav_normalized.wav with 8.소쩍새_106.wav
430o030t.wav_normalized_mixed_11122.wav saved: mix 430o030t.wav_normalized.wav with 5.멧돼지_221.wav
430o030t.wav_normalized_mixed_11123.wav saved: mix 430o030t.wav_normalized.wav with Lion_6.wav
430o030t.wav_normalized_mixed_11124.wav saved: mix 430o030t.wav_normalized.wav with 5.멧돼지_201.wav
203c0a0y.wav_normalized_mixed_11125.wav saved: mix 203c0a0y.wav_normalized.wav with Elephant_1.wav
203c0a0y.wav_normalized_mixed_11126.wav saved: mix 203c0a0y.wav_normalized.wav with 6.까치_83.wav
203c0a0y.wav_normalized_mixed_11127.wav saved: mix 203c0a0y.wav_normalized.wav with 9.참새_180.wav
203c0a0y.wav_normalized_mixed_11128.wav saved: mix 203c0a0y.wav_normalized.wav with 9.참새_189.wav
443s060c.wav_normalized_mixed_11129.wav saved: mix 443s060c.wav_